# 01. Content-Based Recommendations for Netflix Movies

In [2]:
import pandas as pd
import re
import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words as stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

## Loading Data

The used dataset is 'Netflix Movies and TV Shows' from Shivam Bansal, published on Kaggle (can be found [here](https://www.kaggle.com/datasets/shivamb/netflix-shows)).

In [3]:
#Load data
df = pd.read_csv('data/netflix_titles.csv')
#Only keep movies, get rid of TV shows
df = df.loc[df['type'] == 'Movie'].reset_index(drop=True)
#only keep the columns we're interested in
df = df[['title','listed_in','director','cast','description']]
df

,title,listed_in,director,cast,description
0,Dick Johnson Is Dead,Documentaries,Kirsten Johnson,NaN,"As her father nears the end of his life, filmm..."
1,My Little Pony: A New Generation,Children & Family Movies,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",Equestria's divided. But a bright-eyed hero be...
2,Sankofa,"Dramas, Independent Movies, International Movies",Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","On a photo shoot in Ghana, an American model s..."
3,The Starling,"Comedies, Dramas",Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",A woman adjusting to life after a loss contend...
4,Je Suis Karl,"Dramas, International Movies",Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...",After most of her family is murdered in a terr...
...,...,...,...,...,...
6126,Zinzana,"Dramas, International Movies, Thrillers",Majid Al Ansari,"Ali Suliman, Saleh Bakri, Yasa, Ali Al-Jabri, ...",Recovering alcoholic Talal wakes up inside a s...
6127,Zodiac,"Cult Movies, Dramas, Thrillers",David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...","A political cartoonist, a crime reporter and a..."
6128,Zombieland,"Comedies, Horror Movies",Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",Looking to survive in a world taken over by zo...
6129,Zoom,"Children & Family Movies, Comedies",Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...","Dragged from civilian life, a former superhero..."


In [4]:
#Check if columns have NaN values
df.isna().sum()

title            0
listed_in        0
director       188
cast           475
description      0
dtype: int64

In [5]:
#Uncomment to delete rows with NaN values
#df = df.dropna().reset_index(drop=True)

## Recommendations based on the movie descriptions
As a first step, I wanted to see how the recommendations would turn out if only the descriptions were used to generate them.

### TFIDF
To do this, I needed some kind of numerical representation of the film descriptions so that I can compare them. I decided to use TF-IDF to achieve this.

In [6]:
#Define Tokeniser
lem = WordNetLemmatizer()
def my_tokeniser(doc):
    tokens = re.split(r'[-\s.,;!?:()]+', doc)
    return [lem.lemmatize(t.lower()) for t in tokens if not t.lower() in stop_words.ENGLISH_STOP_WORDS]

In [7]:
#Get TFIDF vectors
tfidf_vectoriser = TfidfVectorizer(tokenizer=my_tokeniser)
tfidf = tfidf_vectoriser.fit_transform(df["description"])
tfidf.shape

(6131, 14902)

### Get Recommendations

In [8]:
#Calculate cosine similarity between all movies
similarities = cosine(tfidf)
similarities = pd.DataFrame(similarities, columns = df["title"], index = df["title"])
similarities.shape

(6131, 6131)

In [9]:
pd.set_option("display.max_colwidth", None) #to display the full movie descriptions later on

#Function to get recommendations and corresponding descriptions
def get_recommendations (title, n):
    #get the recommendations
    recommendation = similarities.sort_values(by = title, ascending=False)[title].index[1:n+1]
    #print description of the seed movie
    print('SEED MOVIE DESCRIPTION:',df.loc[df['title'] == title][['description']])
    #get titles & descriptions from df for the recommended movie titles
    df_recommendation = df.loc[df['title'].isin(recommendation)][['title','description', 'listed_in']]
    #sort titles by order of recommendation (so that the first title in the displayed df is also the first recommendation)
    df_recommendation['title'] = pd.Categorical(df_recommendation['title'], recommendation)
    print('\nRECOMMENDED MOVIES:')
    return df_recommendation.sort_values("title") 

#### For Kung Fu Panda

In [10]:
get_recommendations("Kung Fu Panda", 5)

SEED MOVIE DESCRIPTION:                                                                                                                                            description
364  When a powerful villain comes after peace in his valley, a lazy panda rises to the challenge to realize his destiny and become a kung fu warrior.

RECOMMENDED MOVIES:


,title,description,listed_in
4961,Love on Delivery,"To prove himself to a love interest, a delivery boy learns dubious kung fu from an aging charlatan so he can challenge his archrival to a fight.","Action & Adventure, Comedies, International Movies"
4883,Kung Fu Magoo,"Shortsighted curmudgeon Mr. Magoo and his young nephew, Justin, fight such nefarious foes as kung fu villains, robot spiders and teenage mutants.","Children & Family Movies, Comedies"
5719,The Five Venoms,A dying kung fu master sends his last student to track down five former pupils in order to learn which one of them has become evil.,"Action & Adventure, Cult Movies, International Movies"
3777,Kung Fu Panda: Secrets of the Scroll,"When a twist of fate brings five unlikely animals together, they find that combining their unique kung fu styles as a team makes the perfect weapon!","Children & Family Movies, Comedies"
4993,Man of Tai Chi,"In this kung fu tale set in Beijing, a young man armed with incomparable martial arts skills gets embroiled in the world of underground fight clubs.","Action & Adventure, Dramas"


All the recommended movies have to do with Kung Fu in some way, and this theme has definitely been correctly identified. However, "Kung Fu Panda" is an animated film for children, thus "Kung Fu Magoo" and "Kung Fu Panda: Secrets of the Scroll" are good recommendations, while the other three films are action films that are more aimed towards adults.

#### For Charlie and the Chocolate Factory:

In [11]:
get_recommendations("Charlie and the Chocolate Factory", 5)

SEED MOVIE DESCRIPTION:                                                                                                                                   description
4262  The eccentric Willy Wonka opens the doors of his candy factory to five lucky kids who learn the secrets behind his amazing confections.

RECOMMENDED MOVIES:


,title,description,listed_in
6082,Willy Wonka & the Chocolate Factory,Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.,"Children & Family Movies, Classic Movies, Comedies"
5479,Shorgul,A Hindu boy's feelings for a Muslim girl expose the tensions between their two communities and open the door to chaos and violence.,"Dramas, International Movies"
4137,Best Neighbors,"A just-married couple moves into their new home, finding themselves at war with the kids next door, who seem determined to make their lives miserable.","Comedies, International Movies"
1981,A Ghost Story,"Following a fatal car crash, a man's spirit remains stuck in the home he shared with his wife as doors into the past and future begin to open.","Dramas, Independent Movies"
3216,Dil Se,A radio broadcaster's intense attraction to a mysterious woman opens the door to a tidal wave of powerful emotions and desires.,"Dramas, International Movies, Romantic Movies"


Here you can see the disadvantages of this method: "Shorgul" was probably recommended because the words "open" and "door" appear in its description, as they do in the seed film. Otherwise, these two films have nothing in common. The same applies to the other films; only "Willy Wonka and the Chocolate Factory" is a good recommendation.

#### For The Lord of the Rings

In [12]:
get_recommendations('The Lord of the Rings: The Two Towers', 5)

SEED MOVIE DESCRIPTION:                                                                                                                                        description
5801  Frodo and Sam head to Mordor to destroy the One Ring of Power while Gimli, Legolas and Aragorn search for the orc-captured Merry and Pippin.

RECOMMENDED MOVIES:


,title,description,listed_in
3565,Elaan,"Journalist Naresh is hired to probe illegal activities on an island. When he's captured, a fellow prisoner gives him a ring with magic powers.","Action & Adventure, Cult Movies, International Movies"
5959,Tom and Jerry: The Magic Ring,"When a young wizard leaves Tom to guard his priceless magic ring, Jerry gets the ring stuck on his head, igniting a series of slapstick antics.","Children & Family Movies, Comedies"
5800,The Lord of the Rings: The Return of the King,"Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor.","Action & Adventure, Sci-Fi & Fantasy"
230,The Losers,"After a search-and-destroy mission leads to betrayal, an elite black ops squad seeks revenge. But they must remain deep undercover to even the score.",Action & Adventure
3305,Jab Harry Met Sejal,A philandering tour guide in Europe begins to develop feelings for one of his clients while helping her search for her lost engagement ring.,"Comedies, International Movies, Romantic Movies"


We can observe the same phenomenon as before: For example, it seems like "Elaan" was only recommended because the description includes the words "ring" and "power".

"The Lord of the Rings: The Return of the King" is the only recommendation in this list that actually makes sense.

It would be interesting to see if this behaviour (two words evoking a similarity even though the story is completely different) changes if we use a larger body of text, e.g. the movie subtitles. Since there would then be more words to compare, we would probably get better recommendations.

So I decided to download the corresponding subtitles and explore this in a second notebook. But first, let's try a few more things with the dataset we already have.

___

## Recommendations based on other properties (genre, director and cast)
Let's see how the recommendations change when we use the mentioned properties.

In [13]:
#replace NaN values in df with empty strings
df_new = df.fillna('')

In [14]:
## The code in this cell is from Niharika Pandit (2021), 'Netflix Visualizations, Recommendation, EDA', Kaggle, link: https://www.kaggle.com/code/niharika41298/netflix-visualizations-recommendation-eda#Recommendation-System-(Content-Based)
## slightly modified to fit my selected features

#Make all parameters lowercase & remove spaces between words 
# -> e.g. Vanessa Hudgens becomes vanessahudgens and Vannessa Kirby becomes vannessakirby so that their first name is not registered as the same word
def clean_data(x):
        return str.lower(x.replace(" ", "").replace(","," ")) #modified this line to also replace commas with spaces
features=['listed_in','director','cast']
for feature in features:
    df_new[feature] = df_new[feature].apply(clean_data)
    
df_new.head(3)

,title,listed_in,director,cast,description
0,Dick Johnson Is Dead,documentaries,kirstenjohnson,,"As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable."
1,My Little Pony: A New Generation,children&familymovies,robertcullen joséluisucha,vanessahudgens kimikoglenn jamesmarsden sofiacarson lizakoshy kenjeong elizabethperkins janekrakowski michaelmckean phillamarr,"Equestria's divided. But a bright-eyed hero believes Earth Ponies, Pegasi and Unicorns should be pals — and, hoof to heart, she’s determined to prove it."
2,Sankofa,dramas independentmovies internationalmovies,hailegerima,kofighanaba oyafunmikeogunlano alexandraduah nickmedley mutabaruka afemoomilami reggiecarter mzuri,"On a photo shoot in Ghana, an American model slips back in time, becomes enslaved on a plantation and bears witness to the agony of her ancestral past."


I used Bag-of-Words for this, as we only need to know how often a certain word occurs. TF-IDF, which we used before, also tells us how important a word is in a document compared to the rest of the corpus, but we don't need that for the genre/director/cast, as everything should have the same importance.

In [15]:
## The code in this cell is from Niharika Pandit (2021), 'Netflix Visualizations, Recommendation, EDA', Kaggle, link: https://www.kaggle.com/code/niharika41298/netflix-visualizations-recommendation-eda#Recommendation-System-(Content-Based)

#combine all features into one column
def combined(x):
    return x['director'] + ' ' + x['cast'] + ' ' +x['listed_in']

df_new['combined'] = df_new.apply(combined, axis=1)

#Using CountVectorizer to get bag of words
vectoriser = CountVectorizer(stop_words='english')
bag_of_words = vectoriser.fit_transform(df_new['combined'])
bag_of_words.shape

(6131, 31139)

In [16]:
#Calculate cosine similarity
similarities_features = cosine(bag_of_words)
similarities_features = pd.DataFrame(similarities_features, columns = df["title"], index = df["title"])
similarities_features.shape

(6131, 6131)

### Get Recommendations

In [17]:
def get_recommendations_new (title, n, similarities):
    recommendation = similarities.sort_values(by = title, ascending=False)[title].index[1:n+1]
    df_recommendation = df.loc[df['title'].isin(recommendation)]
    df_recommendation['title'] = pd.Categorical(df_recommendation['title'], recommendation)
    return df_recommendation.sort_values("title")

#### For Kung Fu Panda

In [18]:
get_recommendations_new('Kung Fu Panda', 5, similarities_features)

,title,listed_in,director,cast,description
4884,Kung Fu Panda: Holiday,"Children & Family Movies, Comedies",Tim Johnson,"Jack Black, Angelina Jolie, Dustin Hoffman, Jackie Chan, Seth Rogen, Lucy Liu, David Cross, James Hong, Dan Fogler","As preparations for the Winter Feast build, Po is caught between his obligations as the Dragon Warrior and his family holiday traditions."
365,Kung Fu Panda 2,"Children & Family Movies, Comedies",Jennifer Yuh Nelson,"Jack Black, Angelina Jolie, Dustin Hoffman, Gary Oldman, Seth Rogen, Lucy Liu, Jackie Chan, David Cross, James Hong, Michelle Yeoh","With his fists up and belly full, Po embarks on a perilous journey with the Furious Five to take on a frightful new enemy and save the art of kung fu."
3777,Kung Fu Panda: Secrets of the Scroll,"Children & Family Movies, Comedies",Rodolphe Guenoden,"Jack Black, Dustin Hoffman, Seth Rogen, David Cross, Lucy Liu, Tara Macri, James Hong, Randall Duk Kim","When a twist of fate brings five unlikely animals together, they find that combining their unique kung fu styles as a team makes the perfect weapon!"
658,Keymon and Nani in Space Adventure,"Children & Family Movies, Comedies",NaN,NaN,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!"
1449,Little Singham: Legend of Dugabakka,"Children & Family Movies, Comedies",NaN,NaN,"As fears rise over a ghost known for nighttime abductions, Mirchi Nagar residents look to the only person they know can save them: Little Singham!"


Now basically all the other Kung Fu Panda films are recommended to us because they have the same genres and a similar cast. This makes sense, but perhaps users would like a little more variety.

#### For Charlie and the Chocolate Factory

In [19]:
get_recommendations_new('Charlie and the Chocolate Factory', 5, similarities_features)

,title,listed_in,director,cast,description
658,Keymon and Nani in Space Adventure,"Children & Family Movies, Comedies",NaN,NaN,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!"
1449,Little Singham: Legend of Dugabakka,"Children & Family Movies, Comedies",NaN,NaN,"As fears rise over a ghost known for nighttime abductions, Mirchi Nagar residents look to the only person they know can save them: Little Singham!"
4940,Little Singham aur Kaal ka Mahajaal,"Children & Family Movies, Comedies",Prakash Satam,NaN,"When Kaal, the vilest demon on earth, threatens to destroy his village, child cop Little Singham must use his strength and smarts to fight back."
2189,Little Singham: Kaal Ki Tabaahi,"Children & Family Movies, Comedies",Prakash Satam,NaN,Kaal is back – bigger and badder than ever! Can Little Singham fight the battle of a lifetime to stop the demon’s plans to wreak havoc on earth?
4942,Little Singham in London,"Children & Family Movies, Comedies",Prakash Satam,NaN,"Little Singham is in London to meet the queen, but when the famed Kohinoor Diamond gets stolen, the kid cop goes on a wild, citywide hunt for the thief."


From a quick look at the dates in Excel, I already knew that "Charlie and the Chocolate Factory" and "Sweeney Todd: The Demon Barber of Fleet Street" had the same director (Tim Burton) and actor (Johnny Depp) - so I was surprised when I didn't see the movie in the generated recommendations.

That's why I decided to look at the top recommended film ('Keymon and Nani in Space Adventure') in comparison to 'Sweeney Todd'.

In [20]:
df.loc[(df['title'] == 'Charlie and the Chocolate Factory') | (df['title'] == 'Sweeney Todd: The Demon Barber of Fleet Street')| (df['title'] == 'Keymon and Nani in Space Adventure')]

,title,listed_in,director,cast,description
658,Keymon and Nani in Space Adventure,"Children & Family Movies, Comedies",NaN,NaN,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!"
4262,Charlie and the Chocolate Factory,"Children & Family Movies, Comedies",Tim Burton,"Johnny Depp, Freddie Highmore, David Kelly, Helena Bonham Carter, Noah Taylor, Missi Pyle, James Fox, Deep Roy, Christopher Lee, Adam Godley, AnnaSophia Robb, Julia Winter, Jordan Fry, Philip Wiegratz",The eccentric Willy Wonka opens the doors of his candy factory to five lucky kids who learn the secrets behind his amazing confections.
5584,Sweeney Todd: The Demon Barber of Fleet Street,"Dramas, Horror Movies, Music & Musicals",Tim Burton,"Johnny Depp, Helena Bonham Carter, Alan Rickman, Timothy Spall, Sacha Baron Cohen, Jamie Campbell Bower, Laura Michelle Kelly, Jayne Wisener, Ed Sanders","After being falsely imprisoned, a vengeful murderer covers his tracks by enlisting the help of a baker whose meat pies become the toast of London."


In [21]:
#Cosine similarity between Keymon and Nani & Charlie
similarities_features.loc['Keymon and Nani in Space Adventure']['Charlie and the Chocolate Factory']

0.40824829046386313

In [22]:
#Cosine similarity between Sweeney Todd & Charlie
similarities_features.loc['Sweeney Todd: The Demon Barber of Fleet Street']['Charlie and the Chocolate Factory']

0.18898223650461363

-> Both films have two things in common with 'Charlie and the Chocolate Factory':

- Little Singham: The same genres (Children & Family Movies + Comedies)
- Sweeney Todd: Tim Burton + Johnny Depp

'Little Singham' has a higher cosine similarity because it has no values for cast and director. This gives it more similarity to 'Charlie and the Chocolate Factory', while 'Sweeney Todd' lists more actors and therefore has more differences.

A simple solution to avoid this problem would be to delete all movies with NaN values form the dataset, which can be done by uncommenting the code at the very beginning of this notebook.

#### For The Lord of the Rings

In [23]:
get_recommendations_new('The Lord of the Rings: The Two Towers', 5, similarities_features)

,title,listed_in,director,cast,description
5800,The Lord of the Rings: The Return of the King,"Action & Adventure, Sci-Fi & Fantasy",Peter Jackson,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mortensen, Sean Astin, Cate Blanchett, John Rhys-Davies, Bernard Hill, Billy Boyd, Dominic Monaghan, Orlando Bloom, Hugo Weaving","Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor."
5443,Season of the Witch,"Action & Adventure, Sci-Fi & Fantasy",Dominic Sena,"Nicolas Cage, Ron Perlman, Christopher Lee, Stephen Campbell Moore, Stephen Graham, Ulrich Thomsen, Claire Foy, Robert Sheehan, Kevin Rees, Andrew Hefler",A group of weary warriors transport a suspected witch believed to be responsible for spreading the devastating Black Plague.
4747,Indiana Jones and the Kingdom of the Crystal Skull,"Action & Adventure, Children & Family Movies, Sci-Fi & Fantasy",Steven Spielberg,"Harrison Ford, Cate Blanchett, Karen Allen, Ray Winstone, John Hurt, Jim Broadbent, Igor Jijikine, Shia LaBeouf","Indiana Jones is drawn into a Russian plot to unearth the Crystal Skulls of Akator, Amazonian artifacts with supernatural powers."
3857,9,"Action & Adventure, Independent Movies, Sci-Fi & Fantasy",Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly, Christopher Plummer, Crispin Glover, Martin Landau, Fred Tatasciore, Alan Oppenheimer, Tom Kane","In a postapocalyptic world, rag-doll robots hide in fear from dangerous machines out to exterminate them, until a brave newcomer joins the group."
5013,Marvel's Iron Man & Hulk: Heroes United,"Action & Adventure, Sci-Fi & Fantasy","Leo Riley, Eric Radomski","Adrian Pasdar, Fred Tatasciore, Dee Bradley Baker, Robin Atkin Downes, David Kaye","When scientists join Hulk's gamma energy with Tony Stark's Arc Reactor, the resulting force creates a foe that only the two Marvel heroes can combat."


In [24]:
#Look at the properties for the seed movie
df.loc[df['title'] == "The Lord of the Rings: The Two Towers"]

,title,listed_in,director,cast,description
5801,The Lord of the Rings: The Two Towers,"Action & Adventure, Sci-Fi & Fantasy",Peter Jackson,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mortensen, Sean Astin, Cate Blanchett, John Rhys-Davies, Bernard Hill, Christopher Lee, Billy Boyd, Dominic Monaghan, Orlando Bloom","Frodo and Sam head to Mordor to destroy the One Ring of Power while Gimli, Legolas and Aragorn search for the orc-captured Merry and Pippin."


- "The Lord of the Rings: The Return of the King" is recommended because it has the same genre, the same director and mostly the same actors. 
- "Season of the Witch" has the same genres and Christopher Lee is also an actor in it. 
- "Indiana Jones" has Cate Blanchett as an actor in common, but one genre (Children & Family movies) more.

I would say "Season of the Witches" is a recommendation that makes sense. The other films are not completely off the mark, but they're not the best recommendations either.

____
## Recommendations for descriptions + properties
Let's combine both approaches.

I also decided to add a weighting variable for the descriptions so that a user could theoretically influence how important the descriptions should be for the recommendations.

In [25]:
description_weight = 2 #control the weight the descriptions have in comparison to director, cast & genre

#Combine both cosine similarity dataframes
create_sum = lambda s1, s2: s1 + s2
total_similarities = similarities_features.combine((similarities*description_weight), create_sum)

### Get Recommendations
As expected, the results are mostly a mixture of the recommendations we have already seen. Overall, this combination seems to give the best results.

#### For Kung Fu Panda

In [26]:
get_recommendations_new("Kung Fu Panda", 5, total_similarities)

,title,listed_in,director,cast,description
3777,Kung Fu Panda: Secrets of the Scroll,"Children & Family Movies, Comedies",Rodolphe Guenoden,"Jack Black, Dustin Hoffman, Seth Rogen, David Cross, Lucy Liu, Tara Macri, James Hong, Randall Duk Kim","When a twist of fate brings five unlikely animals together, they find that combining their unique kung fu styles as a team makes the perfect weapon!"
365,Kung Fu Panda 2,"Children & Family Movies, Comedies",Jennifer Yuh Nelson,"Jack Black, Angelina Jolie, Dustin Hoffman, Gary Oldman, Seth Rogen, Lucy Liu, Jackie Chan, David Cross, James Hong, Michelle Yeoh","With his fists up and belly full, Po embarks on a perilous journey with the Furious Five to take on a frightful new enemy and save the art of kung fu."
4884,Kung Fu Panda: Holiday,"Children & Family Movies, Comedies",Tim Johnson,"Jack Black, Angelina Jolie, Dustin Hoffman, Jackie Chan, Seth Rogen, Lucy Liu, David Cross, James Hong, Dan Fogler","As preparations for the Winter Feast build, Po is caught between his obligations as the Dragon Warrior and his family holiday traditions."
4883,Kung Fu Magoo,"Children & Family Movies, Comedies",Andrés Couturier,"Dylan Sprouse, Cole Sprouse, Alyson Stoner, Tom Kenny, Chris Parnell, Rodger Bumpass, Maile Flanagan, Wally Wingert, Candi Milo, Jim Ward","Shortsighted curmudgeon Mr. Magoo and his young nephew, Justin, fight such nefarious foes as kung fu villains, robot spiders and teenage mutants."
1449,Little Singham: Legend of Dugabakka,"Children & Family Movies, Comedies",NaN,NaN,"As fears rise over a ghost known for nighttime abductions, Mirchi Nagar residents look to the only person they know can save them: Little Singham!"


#### For Charlie and the Chocolate Factory

In [27]:
get_recommendations_new('Charlie and the Chocolate Factory', 5, total_similarities)

,title,listed_in,director,cast,description
6082,Willy Wonka & the Chocolate Factory,"Children & Family Movies, Classic Movies, Comedies",Mel Stuart,"Gene Wilder, Jack Albertson, Peter Ostrum, Roy Kinnear, Julie Dawn Cole, Leonard Stone, Denise Nickerson, Dodo Denney, Paris Themmen, Ursula Reit",Zany Willy Wonka causes a stir when he announces that golden tickets hidden inside his candy bars will admit holders into his secret confectionary.
4942,Little Singham in London,"Children & Family Movies, Comedies",Prakash Satam,NaN,"Little Singham is in London to meet the queen, but when the famed Kohinoor Diamond gets stolen, the kid cop goes on a wild, citywide hunt for the thief."
658,Keymon and Nani in Space Adventure,"Children & Family Movies, Comedies",NaN,NaN,"For Rohan and his magical pal, Keymon, a trip to visit their grandmother turns into the rescue mission of a lifetime when she is kidnapped by aliens!"
1449,Little Singham: Legend of Dugabakka,"Children & Family Movies, Comedies",NaN,NaN,"As fears rise over a ghost known for nighttime abductions, Mirchi Nagar residents look to the only person they know can save them: Little Singham!"
1419,Sugar High,Children & Family Movies,Ariel Boles,Hunter March,"Talented sugar artists compete for $10,000 over two rounds of competition — candy and sugar sculpture — in this ""Sugar Rush"" spinoff special."


#### For The Lord of the Rings

In [28]:
get_recommendations_new('The Lord of the Rings: The Two Towers', 5, total_similarities)

,title,listed_in,director,cast,description
5800,The Lord of the Rings: The Return of the King,"Action & Adventure, Sci-Fi & Fantasy",Peter Jackson,"Elijah Wood, Ian McKellen, Liv Tyler, Viggo Mortensen, Sean Astin, Cate Blanchett, John Rhys-Davies, Bernard Hill, Billy Boyd, Dominic Monaghan, Orlando Bloom, Hugo Weaving","Aragorn is revealed as the heir to the ancient kings as he, Gandalf and the other members of the broken fellowship struggle to save Gondor."
85,Green Lantern,"Action & Adventure, Sci-Fi & Fantasy",Martin Campbell,"Ryan Reynolds, Blake Lively, Peter Sarsgaard, Mark Strong, Tim Robbins, Jay O. Sanders, Taika Waititi, Angela Bassett",Test pilot Hal Jordan harnesses glowing new powers for good when he wears an otherworldly ring and helps an intergalactic force stop a powerful threat.
3289,Bright,"Action & Adventure, Sci-Fi & Fantasy",David Ayer,"Will Smith, Joel Edgerton, Noomi Rapace, Lucy Fry, Edgar Ramírez, Ike Barinholtz, Brad William Henke, Enrique Murciano, Happy Anderson, Kenneth Choi, Andrea Navedo, Dawn Olivieri","In an LA rife with interspecies tensions, a human cop and his orc partner stumble on a powerful object and become embroiled in a prophesied turf war."
3565,Elaan,"Action & Adventure, Cult Movies, International Movies",K. Ramanlal,"Vinod Mehra, Rekha, Vinod Khanna, Helen, Madan Puri, Brahm Bhardwaj","Journalist Naresh is hired to probe illegal activities on an island. When he's captured, a fellow prisoner gives him a ring with magic powers."
4747,Indiana Jones and the Kingdom of the Crystal Skull,"Action & Adventure, Children & Family Movies, Sci-Fi & Fantasy",Steven Spielberg,"Harrison Ford, Cate Blanchett, Karen Allen, Ray Winstone, John Hurt, Jim Broadbent, Igor Jijikine, Shia LaBeouf","Indiana Jones is drawn into a Russian plot to unearth the Crystal Skulls of Akator, Amazonian artifacts with supernatural powers."
